# **GAN 활용 코드 과제**
---
- 출제자 : 장현빈
- 기수/이름 :

pre-trained 된 모델과 CLIP 모델을 이용하여 StyleGAN을 실행시켜보고,\
image to image translation이 이루어지는 과정을 확인해봅시다!

##**StyleGAN image to image translation 실습**

###**라이브러리 임포트**

In [ ]:
!git clone https://github.com/NVlabs/stylegan3 -q
!git clone https://github.com/openai/CLIP -q
!pip install -e ./CLIP -q
!pip install einops ninja -q

In [ ]:
import sys
sys.path.append('./CLIP')
sys.path.append('./stylegan3')

import io
import os, time
import pickle
import shutil
import numpy as np
from PIL import Image
import torch
import torch.nn.functional as F
import requests
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
import clip
from tqdm.notebook import tqdm
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from IPython.display import display
from einops import rearrange
from google.colab import files

In [ ]:
device = torch.device('cuda')
print('Using device:', device, file=sys.stderr)

###**CLIP 모델로 image embedding 함수 정의**

In [ ]:
def fetch(url_or_path):
    if str(url_or_path).startswith('http://') or str(url_or_path).startswith('https://'):
        r = requests.get(url_or_path)
        r.raise_for_status()
        fd = io.BytesIO()
        fd.write(r.content)
        fd.seek(0)
        return fd
    return open(url_or_path, 'rb')

def fetch_model(url_or_path):
    basename = os.path.basename(url_or_path)
    if os.path.exists(basename):
        return basename
    else:
        !wget -c '{url_or_path}'
        return basename

def norm1(prompt):
    "Normalize to the unit sphere."
    return prompt / prompt.square().sum(dim=-1,keepdim=True).sqrt()

def spherical_dist_loss(x, y):
    x = F.normalize(x, dim=-1)
    y = F.normalize(y, dim=-1)
    return (x - y).norm(dim=-1).div(2).arcsin().pow(2).mul(2)

class MakeCutouts(torch.nn.Module):
    def __init__(self, cut_size, cutn, cut_pow=1.):
        super().__init__()
        self.cut_size = cut_size
        self.cutn = cutn
        self.cut_pow = cut_pow

    def forward(self, input):
        sideY, sideX = input.shape[2:4]
        max_size = min(sideX, sideY)
        min_size = min(sideX, sideY, self.cut_size)
        cutouts = []
        for _ in range(self.cutn):
            size = int(torch.rand([])**self.cut_pow * (max_size - min_size) + min_size)
            offsetx = torch.randint(0, sideX - size + 1, ())
            offsety = torch.randint(0, sideY - size + 1, ())
            cutout = input[:, :, offsety:offsety + size, offsetx:offsetx + size]
            cutouts.append(F.adaptive_avg_pool2d(cutout, self.cut_size))
        return torch.cat(cutouts)

make_cutouts = MakeCutouts(224, 32, 0.5)

def embed_image(image):
  n = image.shape[0]
  cutouts = make_cutouts(image)
  embeds = clip_model.embed_cutout(cutouts)
  embeds = rearrange(embeds, '(cc n) c -> cc n c', n=n)
  return embeds

def embed_url(url):
  image = Image.open(fetch(url)).convert('RGB')
  return embed_image(TF.to_tensor(image).to(device).unsqueeze(0)).mean(0).squeeze(0)

class CLIP(object):
  def __init__(self):
    clip_model = "ViT-B/32"
    self.model, _ = clip.load(clip_model)
    self.model = self.model.requires_grad_(False)
    self.normalize = transforms.Normalize(mean=[0.48145466, 0.4578275, 0.40821073],
                                          std=[0.26862954, 0.26130258, 0.27577711])

  @torch.no_grad()
  def embed_text(self, prompt):
      "Normalized clip text embedding."
      return norm1(self.model.encode_text(clip.tokenize(prompt).to(device)).float())

  def embed_cutout(self, image):
      "Normalized clip image embedding."
      return norm1(self.model.encode_image(self.normalize(image)))

clip_model = CLIP()

###**사용할 pre-trained styleGAN 모델 선택하기**

**아래의 pre-trained model 중 하나를 선택하여, *'model_name'*에 입력해주세요!**

1) ffhq: 얼굴 사진으로 pre-trained
- "stylegan3-r-ffhq-1024x1024.pkl"
- "stylegan3-t-ffhq-1024x1024.pkl"

2) metfaces: 명화 pre-trained
- "stylegan3-r-metfaces-1024x1024.pkl"
- "stylegan3-t-metfaces-1024x1024.pkl"

3) afhqv2: 동물 pre-trained

- "stylegan3-r-afhqv2-512x512.pkl"
- "stylegan3-t-afhqv2-512x512.pkl"


In [ ]:
base_url = "https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/"

model_name = "stylegan3-r-ffhq-1024x1024.pkl"
network_url = base_url + model_name

with open(fetch_model(network_url), 'rb') as fp:
  G = pickle.load(fp)['G_ema'].to(device)

clip_model = CLIP()

###**파라미터 설정**

*text = '???'* 변수에 단어, 구 등을 작성합니다.\
ex. 유명 인물 이름, 캐릭터 이름, 동물 이름 등
- pre-trained 모델로 선택한 종류를 기반으로 작성해야 더 좋은 이미지가 생성됩니다!
- 사용할 모델을 변경하려는 경우 위의 모델 선택 코드를 다시 수행해주셔야 합니다!


*steps*: epoch 수 지정\
*show_every_n_steps*: epoch 몇 번 당 한 번 씩 생성 image를 출력할지 지정

In [ ]:
text = 'Barack Obama' # 이걸 바꿔주면 됩니다!
target = clip_model.embed_text(text)

steps = 100

target_images = "em.jpg"

seed =  -1  #-1 for completly random
if seed == -1:
    seed = np.random.randint(0,2**32 - 1)

show_every_n_steps = 20

zs = torch.randn([10000, G.mapping.z_dim], device=device)
w_stds = G.mapping(zs, None).std(0)

fix_coordinates = "True"
if fix_coordinates == "True":
  shift = G.synthesis.input.affine(G.mapping.w_avg.unsqueeze(0))
  G.synthesis.input.affine.bias.data.add_(shift.squeeze(0))
  G.synthesis.input.affine.weight.data.zero_()

###**Run the Model !**

In [ ]:
from IPython.display import display

tf = Compose([
  Resize(224),
  lambda x: torch.clamp((x+1)/2,min=0,max=1),
  ])

def run(seed, G):
  torch.manual_seed(seed)
  timestring = time.strftime('%Y%m%d%H%M%S')
  rand_z = torch.from_numpy(np.random.RandomState(seed).randn(1, G.mapping.z_dim)).to(device)
  q = (G.mapping(rand_z, None, truncation_psi=0.2)) / w_stds
  q.requires_grad_()

  with torch.no_grad():
    qs = []
    losses = []
    for _ in range(8):
      q = (G.mapping(torch.randn([4,G.mapping.z_dim], device=device), None, truncation_psi=0.7) - G.mapping.w_avg) / w_stds
      images = G.synthesis(q * w_stds + G.mapping.w_avg)
      embeds = embed_image(images.add(1).div(2))
      loss = spherical_dist_loss(embeds, target).mean(0)
      i = torch.argmin(loss)
      qs.append(q[i])
      losses.append(loss[i])
    qs = torch.stack(qs)
    losses = torch.stack(losses)
    i = torch.argmin(losses)
    q = qs[i].unsqueeze(0).requires_grad_()

# Sampling loop
  q_ema = q
  opt = torch.optim.AdamW([q], lr=0.03, betas=(0.0,0.999))
  loop = tqdm(range(steps))
  for i in loop:
    opt.zero_grad()
    w = q * w_stds
    image = G.synthesis(w + G.mapping.w_avg, noise_mode='const')
    embed = embed_image(image.add(1).div(2))
    loss = spherical_dist_loss(embed, target).mean()
    loss.backward()
    opt.step()
    loop.set_postfix(loss=loss.item(), q_magnitude=q.std().item())

    q_ema = q_ema * 0.9 + q * 0.1
    image = G.synthesis(q_ema * w_stds + G.mapping.w_avg, noise_mode='const')

    if i % 10 == 0:
      display(TF.to_pil_image(tf(image)[0]))
    pil_image = TF.to_pil_image(image[0].add(1).div(2).clamp(0,1))
    os.makedirs(f'samples/{timestring}', exist_ok=True)
    pil_image.save(f'samples/{timestring}/{i:04}.jpg')

  # Save images as a tar archive
  !tar cf samples/{timestring}.tar samples/{timestring}

  return timestring

timestring = run(seed, G)

In [ ]:
base_url = "https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/"

model_name = "stylegan3-r-afhqv2-512x512.pkl"
network_url = base_url + model_name

with open(fetch_model(network_url), 'rb') as fp:
  G = pickle.load(fp)['G_ema'].to(device)

clip_model = CLIP()

In [ ]:
text = 'Rhinoceros' # 이걸 바꿔줘도 됩니다!
target = clip_model.embed_text(text)

steps = 100

target_images = "em.jpg"

seed =  -1  #-1 for completly random
if seed == -1:
    seed = np.random.randint(0,2**32 - 1)

show_every_n_steps = 20

zs = torch.randn([10000, G.mapping.z_dim], device=device)
w_stds = G.mapping(zs, None).std(0)

fix_coordinates = "True"
if fix_coordinates == "True":
  shift = G.synthesis.input.affine(G.mapping.w_avg.unsqueeze(0))
  G.synthesis.input.affine.bias.data.add_(shift.squeeze(0))
  G.synthesis.input.affine.weight.data.zero_()

In [ ]:
from IPython.display import display

tf = Compose([
  Resize(224),
  lambda x: torch.clamp((x+1)/2,min=0,max=1),
  ])

def run(seed, G):
  torch.manual_seed(seed)
  timestring = time.strftime('%Y%m%d%H%M%S')
  rand_z = torch.from_numpy(np.random.RandomState(seed).randn(1, G.mapping.z_dim)).to(device)
  q = (G.mapping(rand_z, None, truncation_psi=0.2)) / w_stds
  q.requires_grad_()

  with torch.no_grad():
    qs = []
    losses = []
    for _ in range(8):
      q = (G.mapping(torch.randn([4,G.mapping.z_dim], device=device), None, truncation_psi=0.7) - G.mapping.w_avg) / w_stds
      images = G.synthesis(q * w_stds + G.mapping.w_avg)
      embeds = embed_image(images.add(1).div(2))
      loss = spherical_dist_loss(embeds, target).mean(0)
      i = torch.argmin(loss)
      qs.append(q[i])
      losses.append(loss[i])
    qs = torch.stack(qs)
    losses = torch.stack(losses)
    i = torch.argmin(losses)
    q = qs[i].unsqueeze(0).requires_grad_()

# Sampling loop
  q_ema = q
  opt = torch.optim.AdamW([q], lr=0.03, betas=(0.0,0.999))
  loop = tqdm(range(steps))
  for i in loop:
    opt.zero_grad()
    w = q * w_stds
    image = G.synthesis(w + G.mapping.w_avg, noise_mode='const')
    embed = embed_image(image.add(1).div(2))
    loss = spherical_dist_loss(embed, target).mean()
    loss.backward()
    opt.step()
    loop.set_postfix(loss=loss.item(), q_magnitude=q.std().item())

    q_ema = q_ema * 0.9 + q * 0.1
    image = G.synthesis(q_ema * w_stds + G.mapping.w_avg, noise_mode='const')

    if i % 10 == 0:
      display(TF.to_pil_image(tf(image)[0]))
    pil_image = TF.to_pil_image(image[0].add(1).div(2).clamp(0,1))
    os.makedirs(f'samples/{timestring}', exist_ok=True)
    pil_image.save(f'samples/{timestring}/{i:04}.jpg')

  # Save images as a tar archive
  !tar cf samples/{timestring}.tar samples/{timestring}

  return timestring

timestring = run(seed, G)